# Fake News Capstone Project Data Wrangling and Preprocessing

In [1]:
import s3fs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import tldextract
from tqdm.autonotebook import tqdm
tqdm.pandas(desc="progress-bar", leave=False)
import string
import spacy
from spacy.lang import punctuation
import unicodedata  # might need to pip install unicodedate2 on aws sagemaker
from contractions import CONTRACTION_DICT # need to have the contractions.py file copied to your path
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import STOPWORDS
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set(style='darkgrid',palette='Dark2',rc={'figure.figsize':(9,6),'figure.dpi':90})

punctuation = string.punctuation + '”' + '“' + '–' + '““' + "’’" + '”'
stopword = stopwords.words('english')
stopwords = set(STOPWORDS)
wordnet_lemmatizer = WordNetLemmatizer()

<ipython-input-1-3bd910620f49>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:701: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Random seed

In [2]:
seed = 123

## Read in the dataset

In [3]:
df = pd.read_csv('s3://osanusi-auto/capstone_prep.csv')
df.head(5)

,domain,type,content,title,authors
0,nytimes.com,real,"Paul Shanley, an ex-priest convicted of rape i...",Massachusetts: Court Upholds Ex-Priest’s Convi...,The Associated Press
1,nytimes.com,real,A federal district judge who has monitored the...,National Briefing | Southwest: Texas: Easing R...,"Jim Yardley Nyt, Jim Yardley, Nyt"
2,www.washingtonpost.com,real,NEW YORK — CNN and host Jake Tapper have both ...,CNN and Jake Tapper apologize for ‘unacceptabl...,Associated Press
3,nytimes.com,real,Photo\n\nWe’ve seen plenty of unexpected skin ...,Three’s a Trend: Asymmetrical Harnesses,Angela Koh
4,www.yahoo.com,real,By Rory Carroll (Reuters) - A prosecutor in Mi...,Minnesota officer charged with manslaughter in...,Rory Carroll


In [4]:
# checking missing values
def missing_value(df):
    """" Function to calculate the number and percent of missing values in a dataframe"""
    total = df.isnull().sum().sort_values(ascending=False)
    percent = ((df.isnull().sum()/df.isnull().count())*100).sort_values(ascending=False)
    missing_value = pd.concat([total, percent], axis=1, keys=['Total','Percent'])
    return missing_value

In [5]:
missing_value(df)

,Total,Percent
authors,0,0.0
title,0,0.0
content,0,0.0
type,0,0.0
domain,0,0.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090432 entries, 0 to 1090431
Data columns (total 5 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   domain   1090432 non-null  object
 1   type     1090432 non-null  object
 2   content  1090432 non-null  object
 3   title    1090432 non-null  object
 4   authors  1090432 non-null  object
dtypes: object(5)
memory usage: 41.6+ MB


## Clean up the domain and check the unique domains

In [7]:
def extract_domain(url):
    """
    Extract domain name from fld url
    """
    info = tldextract.extract(url)
    return info.domain

In [8]:
df['domain'] = df['domain'].astype(str).apply(extract_domain)
df.head(3)

,domain,type,content,title,authors
0,nytimes,real,"Paul Shanley, an ex-priest convicted of rape i...",Massachusetts: Court Upholds Ex-Priest’s Convi...,The Associated Press
1,nytimes,real,A federal district judge who has monitored the...,National Briefing | Southwest: Texas: Easing R...,"Jim Yardley Nyt, Jim Yardley, Nyt"
2,washingtonpost,real,NEW YORK — CNN and host Jake Tapper have both ...,CNN and Jake Tapper apologize for ‘unacceptabl...,Associated Press


## Cleaning up the content column by removing all the noises

In [9]:
def remove_special_characters(text): 
    """
    Removes special characters from the text document
    """
    # define the pattern to keep. You can check the regex using this url https://regexr.com/
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
    return re.sub(pat, '', text)

def remove_extra_whitespace_tabs(text): 
    """
    Removes extra whitespaces and remove_extra_whitespace_tabs
    """
    #pattern = r'^\s+$|\s+$'
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()

def remove_digits(text): 
    """
    Remove all digits from the text document
     take string input and return a clean text without numbers.
        Use regex to discard the numbers.
    """
    result = ''.join(i for i in text if not i.isdigit()).lower()
    return ' '.join(result.split())

def remove_newlines(text): 
    """
    Remove newline characters from the text document
    """
    return text.replace('\\n', ' ').replace('\\r', ' ').replace('\n', ' ').replace('\r', ' ').replace('\\', ' ')

#normalize to the NFKD (Normalization Form Compatibility Decomposition) form
#that present in the Unicode standard to remain compatible with other encodings
def remove_accented_chars(text): 
    """
    Removes accented characters from the test
    """
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

#expands contractions found in the text
def expand_contractions(text, contraction_mapping=CONTRACTION_DICT):

    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

# replace punctuation characters with spaces
def replace_punctuation(text):
    filters = string.punctuation + '”' + '“' + '–' 
    translate_dict = dict((c, " ") for c in filters)   
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    return text

# Remove stopwords and remove words with 2 or less characters
def stops_letters(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stopword:
            result.append(token)
            
    return " ".join(result)

#Removes any word that starts with either http or https
def remove_urls (vTEXT):
    #vTEXT = re.sub('http://\S+|https://\S+', '', vTEXT,flags=re.MULTILINE)
    vTEXT = re.sub('http[s]?://\S+', '', vTEXT,flags=re.MULTILINE)
    return(vTEXT)
#Remove words that starts with www
def remove_www (vTEXT):
    vTEXT = re.sub('www\S+', '', vTEXT,flags=re.MULTILINE)
    return(vTEXT)


In [10]:
%%time
# Apply the functions to the dataframe

# Step 1 - convert the text to lower case
df['content']=df['content'].apply(lambda x: x.lower())
df['title']=df['title'].apply(lambda x: x.lower())
df['authors']=df['authors'].apply(lambda x: x.lower())

#step 2 - Remove URLS
df['content']=df['content'].apply(remove_urls)
df['title'] = df['title'].apply(remove_urls)

#step 3 - Remove www
df['content']=df['content'].apply(remove_www)
df['title'] = df['title'].apply(remove_www)

# Step 4 - remove special charcaters
df['content']=df['content'].apply(remove_special_characters)
df['title'] = df['title'].apply(remove_special_characters)

#step 5 - Remove whitespaces and tabs
df['content']=df['content'].apply(remove_extra_whitespace_tabs)
df['title'] = df['title'].apply(remove_extra_whitespace_tabs)
df['authors'] = df['authors'].apply(remove_extra_whitespace_tabs)

#step 6 - remove newlines and tabs
df['content'] = df['content'].apply(remove_newlines)
df['title'] = df['title'].apply(remove_newlines)

# step 7 - Remove digits
df['content']=df['content'].apply(remove_digits)
df['title'] = df['title'].apply(remove_digits)

#step 8 - remove accented characters
df['content']=df['content'].apply(remove_accented_chars)
df['title'] = df['title'].apply(remove_accented_chars)

#step 9 - Expand contractions
df['content']=df['content'].apply(expand_contractions)

#step 10 - Replace punctuations with spaces 
df['content']= df['content'].apply(replace_punctuation)
df['title'] = df['title'].apply(replace_punctuation)
df['authors'] = df['authors'].apply(replace_punctuation)

#step 11 - Remove stopwords, tokenize and remove words with 3 letters or less
df['content']= df['content'].apply(stops_letters)
df['title'] = df['title'].apply(stops_letters)
df['authors'] = df['authors'].apply(stops_letters)


Wall time: 3h 4min 56s


## Normalization - Lemmatize the title and content columns

In [11]:
def lemmatized_word(text):
    """
    lemmatize the text so as to get its root form eg: functions,funtionality as function
    """
    word_tokens = nltk.word_tokenize(text)
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]
    return  " ".join(lemmatized_word) #combine the words into a giant string that vectorizer can accept


In [12]:
df['content'] = df['content'].progress_apply(lemmatized_word)
df['title'] = df['title'].progress_apply(lemmatized_word)


progress-bar:   0%|          | 0/1090432 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/1090432 [00:00<?, ?it/s]

## Create additional features

In [13]:
%%time
# word counts
df['c_word_count'] = df["content"].apply(lambda x: len(str(x).split(" ")))
df['t_word_count'] = df["title"].apply(lambda x: len(str(x).split(" ")))

# Character counts
df['c_character_count'] = df["content"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
df['t_character_count'] = df["title"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#average word length
df['c_avg_word_length'] = df['c_character_count'] / df['c_word_count']
df['t_avg_word_length'] = df['t_character_count'] / df['t_word_count']


Wall time: 1min 13s


## Identify sentiment in the title

In [14]:
def sentiment_check (text):
    polarity_score = TextBlob(text).sentiment.polarity
    if polarity_score < 0:
        return 'negative'
    elif polarity_score == 0:
        return 'neutral'
    else:
        return 'positive'

In [15]:
%%time
df['sentiment'] = df['title'].apply(sentiment_check)
df.head(3)

Wall time: 7min 33s


,domain,type,content,title,authors,c_word_count,t_word_count,c_character_count,t_character_count,c_avg_word_length,t_avg_word_length,sentiment
0,nytimes,real,paul shanley expriest convicted rape scandal s...,massachusetts court upholds expriests conviction,associated press,56,5,365,44,6.517857,8.800000,neutral
1,nytimes,real,federal district judge monitored state prison ...,national briefing southwest texas easing rein ...,yardley yardley,45,7,311,46,6.911111,6.571429,neutral
2,washingtonpost,real,york host jake tapper apologized onscreen bann...,jake tapper apologize unacceptable banner text,associated press,82,6,609,41,7.426829,6.833333,neutral


## Rename the target column

In [17]:
df = df.rename({'type': 'label'}, axis=1)  


## Rearranged the order of the columns

In [18]:
df = df[['domain','title','authors','content','c_word_count','t_word_count','c_character_count','t_character_count','c_avg_word_length','t_avg_word_length','sentiment','label']]
df.head(3)

,domain,title,authors,content,c_word_count,t_word_count,c_character_count,t_character_count,c_avg_word_length,t_avg_word_length,sentiment,label
0,nytimes,massachusetts court upholds expriests conviction,associated press,paul shanley expriest convicted rape scandal s...,56,5,365,44,6.517857,8.800000,neutral,real
1,nytimes,national briefing southwest texas easing rein ...,yardley yardley,federal district judge monitored state prison ...,45,7,311,46,6.911111,6.571429,neutral,real
2,washingtonpost,jake tapper apologize unacceptable banner text,associated press,york host jake tapper apologized onscreen bann...,82,6,609,41,7.426829,6.833333,neutral,real


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090432 entries, 0 to 1090431
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   domain             1090432 non-null  object 
 1   title              1090432 non-null  object 
 2   authors            1090432 non-null  object 
 3   content            1090432 non-null  object 
 4   c_word_count       1090432 non-null  int64  
 5   t_word_count       1090432 non-null  int64  
 6   c_character_count  1090432 non-null  int64  
 7   t_character_count  1090432 non-null  int64  
 8   c_avg_word_length  1090432 non-null  float64
 9   t_avg_word_length  1090432 non-null  float64
 10  sentiment          1090432 non-null  object 
 11  label              1090432 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 99.8+ MB


In [20]:
df.head(4)

,domain,title,authors,content,c_word_count,t_word_count,c_character_count,t_character_count,c_avg_word_length,t_avg_word_length,sentiment,label
0,nytimes,massachusetts court upholds expriests conviction,associated press,paul shanley expriest convicted rape scandal s...,56,5,365,44,6.517857,8.800000,neutral,real
1,nytimes,national briefing southwest texas easing rein ...,yardley yardley,federal district judge monitored state prison ...,45,7,311,46,6.911111,6.571429,neutral,real
2,washingtonpost,jake tapper apologize unacceptable banner text,associated press,york host jake tapper apologized onscreen bann...,82,6,609,41,7.426829,6.833333,neutral,real
3,nytimes,three trend asymmetrical harness,angela,photo weve seen plenty unexpected skin london ...,65,4,407,29,6.261538,7.250000,neutral,real


## Saving the preprocessed dataframe to csv

In [21]:
# path to save the preprocessed csv file
file_path = r"C:\Users\lanre\Documents\data_science\capstone-preprocessed.csv"

In [22]:
df.to_csv(file_path, index=False)